<a href="https://colab.research.google.com/github/gpadem/algorhythm_use_case/blob/main/book_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 9.1MB/s 
     |████████████████████████████████| 901kB 47.5MB/s 
     |████████████████████████████████| 3.3MB 55.6MB/s 


In [2]:
from transformers import pipeline

In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
# Around the world in 80 days from Jules Verne
book_url = "https://www.gutenberg.org/files/103/103-h/103-h.htm"

In [6]:
r = requests.get(book_url, 'utf-8')

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'h3', 'p'])
text = [result.text for result in results]
BOOK = ' '.join(text).replace("\r\n", " ")

In [8]:
import re

In [9]:
chapters = re.split("Chapter\s", BOOK)

In [10]:
chapters

 great had been the expense of his tour that, even had he won, it would not have enriched him; and it is probable that he had not sought to enrich himself, being a man who rather laid wagers for honour\'s sake than for the stake proposed.  But this wager totally ruined him.   Mr. Fogg\'s course, however, was fully decided upon; he knew what remained for him to do.   A room in the house in Saville Row was set apart for Aouda, who was overwhelmed with grief at her protector\'s misfortune.  From the words which Mr. Fogg dropped, she saw that he was meditating some serious project.   Knowing that Englishmen governed by a fixed idea sometimes resort to the desperate expedient of suicide, Passepartout kept a narrow watch upon his master, though he carefully concealed the appearance of so doing.   First of all, the worthy fellow had gone up to his room, and had extinguished the gas burner, which had been burning for eighty days. He had found in the letter-box a bill from the gas company, and 

In [11]:
len(chapters)

38

In [13]:
from pandas import DataFrame

In [14]:
df = DataFrame (chapters ,columns=['chapters'])
print (df)

                                             chapters
0             AROUND THE WORLD IN EIGHTY DAYS   By   
1   I   Mr. Phileas Fogg lived, in 1872, at No. 7,...
2   II   "Faith," muttered Passepartout, somewhat ...
3   III   Phileas Fogg, having shut the door of hi...
4   IV   Having won twenty guineas at whist, and t...
5   V   Phileas Fogg rightly suspected that his de...
6   VI   The circumstances under which this telegr...
7   VII   The detective passed down the quay, and ...
8   VIII   Fix soon rejoined Passepartout, who was...
9   IX   The distance between Suez and Aden is pre...
10  X   Everybody knows that the great reversed tr...
11  XI   The train had started punctually.  Among ...
12  XII   In order to shorten the journey, the gui...
13  XIII   The project was a bold one, full of dif...
14  XIV   The rash exploit had been accomplished; ...
15  XV   The train entered the station, and Passep...
16  XVI   The Rangoon—one of the Peninsular and Or...
17  XVII   The detective and

In [19]:
df_chapters = df.drop(labels=0, axis=0)

In [20]:
df_chapters

,chapters
1,"I Mr. Phileas Fogg lived, in 1872, at No. 7,..."
2,"II ""Faith,"" muttered Passepartout, somewhat ..."
3,"III Phileas Fogg, having shut the door of hi..."
4,"IV Having won twenty guineas at whist, and t..."
5,V Phileas Fogg rightly suspected that his de...
6,VI The circumstances under which this telegr...
7,"VII The detective passed down the quay, and ..."
8,"VIII Fix soon rejoined Passepartout, who was..."
9,IX The distance between Suez and Aden is pre...
10,X Everybody knows that the great reversed tr...


In [46]:
max_chunk = 500

In [49]:
summarizer_bart = pipeline('summarization', model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')

RuntimeError: CUDA error: invalid device ordinal

In [38]:
summarizer_t5 = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [39]:
for chapter in df_chapters["chapters"]:
  chapter = chapter.replace('.', '.<eos>')
  chapter = chapter.replace('?', '?<eos>')
  chapter = chapter.replace('!', '!<eos>')

  sentences = chapter.split('<eos>')
  current_chunk = 0 
  chunks = []
  for sentence in sentences:
      if len(chunks) == current_chunk + 1: 
          if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
              chunks[current_chunk].extend(sentence.split(' '))
          else:
              current_chunk += 1
              chunks.append(sentence.split(' '))
      else:
          print(current_chunk)
          chunks.append(sentence.split(' '))

  for chunk_id in range(len(chunks)):
      chunks[chunk_id] = ' '.join(chunks[chunk_id])
  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [40]:
chunks[1]

'   In journeying eastward he had gone towards the sun, and the days therefore diminished for him as many times four minutes as he crossed degrees in this direction.   There are three hundred and sixty degrees on the circumference of the earth; and these three hundred and sixty degrees, multiplied by four minutes, gives precisely twenty-four hours—that is, the day unconsciously gained.   In other words, while Phileas Fogg, going eastward, saw the sun pass the meridian eighty times, his friends in London only saw it pass the meridian seventy-nine times.   This is why they awaited him at the Reform Club on Saturday, and not Sunday, as Mr.  Fogg thought.    And Passepartout\'s famous family watch, which had always kept London time, would have betrayed this fact, if it had marked the days as well as the hours and the minutes!    Phileas Fogg, then, had won the twenty thousand pounds; but, as he had spent nearly nineteen thousand on the way, the pecuniary gain was small.   His object was, h

In [41]:
%%time
res_bart = summarizer_bart(chunks, max_length=500, min_length=300, do_sample=False)

Wall time: 4min 20s


In [42]:
res_bart[0]

{'summary_text': 'Passepartout was sent by his master to engage the services of the Reverend Samuel Wilson in a certain marriage ceremony, which was to take place the next day. He soon reached the clergyman\'s house, but found him not at home. Passepartout waited a good twenty minutes, and when he left the reverend gentleman, it was thirty-five minutes past eight. But in what a state he was!   With his hair in disorder, and without his hat, he ran along the street as never man was seen to run before, overturning passers-by, rushing over the sidewalk like a waterspout. In three minutes he was in Saville Row again, and staggered back into Mr. Fogg\'s room. Phileas Fogg had, without suspecting it, gained one day on his journey, and this merely because he had travelled constantly eastward; he would, on the contrary, have lost a day had he gone in the opposite direction, that is, westward.   How came he to think that he had arrived in London on Saturday, the twenty-first day of December, wh

In [43]:
text_bart = ' '.join([summ['summary_text'] for summ in res_bart])

In [44]:
print(text_bart)

Passepartout was sent by his master to engage the services of the Reverend Samuel Wilson in a certain marriage ceremony, which was to take place the next day. He soon reached the clergyman's house, but found him not at home. Passepartout waited a good twenty minutes, and when he left the reverend gentleman, it was thirty-five minutes past eight. But in what a state he was!   With his hair in disorder, and without his hat, he ran along the street as never man was seen to run before, overturning passers-by, rushing over the sidewalk like a waterspout. In three minutes he was in Saville Row again, and staggered back into Mr. Fogg's room. Phileas Fogg had, without suspecting it, gained one day on his journey, and this merely because he had travelled constantly eastward; he would, on the contrary, have lost a day had he gone in the opposite direction, that is, westward.   How came he to think that he had arrived in London on Saturday, the twenty-first day of December, when it was really Fri

In [40]:
%%time
res_t5 = summarizer_t5(chunks, max_length=200, min_length=100, do_sample=False)

CPU times: user 5min 41s, sys: 11.9 s, total: 5min 53s
Wall time: 3min 51s


In [41]:
res_t5[0]

{'summary_text': "phileas fogg thought he arrived in london on saturday, when it was really friday . fogg had travelled constantly eastward; he would have lost a day if he went westward . frank fogg's errand was to engage the services of a clergyman in a marriage ceremony . the clergyman was not at home, and passepartout waited a good twenty minutes ."}

In [42]:
text_t5 = ' '.join([summ['summary_text'] for summ in res_t5])

In [43]:
print(text_t5)

phileas fogg thought he arrived in london on saturday, when it was really friday . fogg had travelled constantly eastward; he would have lost a day if he went westward . frank fogg's errand was to engage the services of a clergyman in a marriage ceremony . the clergyman was not at home, and passepartout waited a good twenty minutes . phileas fogg, going eastward, saw the sun pass the meridian eighty times . his friends in london only saw it pass 79 times, as he thought . fogg's object was to be victorious, and not to win money . he deducted from Passepartout's share the cost of the gas which had burned in his room . "if you had not suggested our marriage, my servant would not have gone to the reverend Samuel Wilson phileas fogg had won his wager, and had made his journey around the world in eighty days . what had he really gained by all this trouble? nothing but a charming woman, who, strangely, made him the happiest of men . aouda was the most beautiful woman he had ever seen, and he 